# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [1]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
url = 'https://bit.ly/3BPcobU'
table = 'investments'
csize=5000

import sqlite3
import pandas as pd

In [2]:
#
#read the data set into dataframes using 5,000 row chunks


cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
mem_usage = list()

##The total memory footprint of all of the chunks combined.
for c in cbase_iter:
    mem_usage.append(c.memory_usage(deep=True).sum()/(1024*1024))

for i in range(len(mem_usage)):
  print(f"Chunk -- {i} || Memory used -- {mem_usage[i]}_Mb")

print(f"\nTotal memory used by all chunks -- {sum(mem_usage)}_Mb")

Chunk -- 0 || Memory used -- 5.579240798950195_Mb
Chunk -- 1 || Memory used -- 5.528232574462891_Mb
Chunk -- 2 || Memory used -- 5.535050392150879_Mb
Chunk -- 3 || Memory used -- 5.528193473815918_Mb
Chunk -- 4 || Memory used -- 5.5243377685546875_Mb
Chunk -- 5 || Memory used -- 5.553427696228027_Mb
Chunk -- 6 || Memory used -- 5.531436920166016_Mb
Chunk -- 7 || Memory used -- 5.5096588134765625_Mb
Chunk -- 8 || Memory used -- 5.396121025085449_Mb
Chunk -- 9 || Memory used -- 4.639497756958008_Mb
Chunk -- 10 || Memory used -- 2.6637144088745117_Mb

Total memory used by all chunks -- 56.988911628723145_Mb


In [3]:
##Explore column datatypes

cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
chunk= cbase_iter.read(csize)
types = chunk.dtypes
types

company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_name              object
investor_category_code     object
investor_country_code      object
investor_state_code        object
investor_region            object
investor_city              object
funding_round_type         object
funded_at                  object
funded_month               object
funded_quarter             object
funded_year                 int64
raised_amount_usd         float64
dtype: object

In [4]:
#Get list of columns

cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
chunk= cbase_iter.read(csize)
cols = (chunk.columns).values.tolist()
col1=chunk.columns
cols
#col1

['company_permalink',
 'company_name',
 'company_category_code',
 'company_country_code',
 'company_state_code',
 'company_region',
 'company_city',
 'investor_permalink',
 'investor_name',
 'investor_category_code',
 'investor_country_code',
 'investor_state_code',
 'investor_region',
 'investor_city',
 'funding_round_type',
 'funded_at',
 'funded_month',
 'funded_quarter',
 'funded_year',
 'raised_amount_usd']

In [5]:
##Each column's missing value counts and memory footprint ...

def colNan(col):

  cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
  tot=list()
  mem_usage=list()

  for c in cbase_iter:
    chunk_nan = c[col].isnull().sum()
    tot.append(chunk_nan)
    mem_usage.append(c[col].memory_usage(deep=True)/(1024*1024))

  total = sum(tot)
  print(f"\nColumn -- {col} || Total missing values -- {total}")
  print(f"Column -- {col} || Total memory usage -- {sum(mem_usage)}_Mb")
  

##running the function loop over columns
for c in cols:
  colNan(c)


Column -- company_permalink || Total missing values -- 1
Column -- company_permalink || Total memory usage -- 3.8711891174316406_Mb

Column -- company_name || Total missing values -- 1
Column -- company_name || Total memory usage -- 3.4263362884521484_Mb

Column -- company_category_code || Total missing values -- 643
Column -- company_category_code || Total memory usage -- 3.2639999389648438_Mb

Column -- company_country_code || Total missing values -- 1
Column -- company_country_code || Total memory usage -- 3.0266036987304688_Mb

Column -- company_state_code || Total missing values -- 492
Column -- company_state_code || Total memory usage -- 2.9635419845581055_Mb

Column -- company_region || Total missing values -- 1
Column -- company_region || Total memory usage -- 3.2548837661743164_Mb

Column -- company_city || Total missing values -- 533
Column -- company_city || Total memory usage -- 3.3448543548583984_Mb

Column -- investor_permalink || Total missing values -- 2
Column -- inve

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [6]:
# Your code goes here
cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
chunk= cbase_iter.read(csize)
types = chunk.dtypes
types

company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_name              object
investor_category_code     object
investor_country_code      object
investor_state_code        object
investor_region            object
investor_city              object
funding_round_type         object
funded_at                  object
funded_month               object
funded_quarter             object
funded_year                 int64
raised_amount_usd         float64
dtype: object

In [7]:
#check sample data
chunk.head()

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000.0
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000.0
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000.0
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012,20000.0
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011,20000.0


In [8]:
def getUnik(col):

  cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
  vc = list()
  cnt = list()
    
  for ch in cbase_iter:
    chunk_vc = ch[col].value_counts()
    vc.append(chunk_vc)
  
  combined_vc = pd.concat(vc)
  final_vc = combined_vc.groupby(combined_vc.index).sum()
  f=final_vc.sort_values(ascending=False)
  ln = len(f)
  cnt = sum(f<50)
  print (f"Column --{col} || Unique values count = {ln} ")
  #print(f"Unique values count in column ->{col} : {ln} ")
  #print(ln)
  

##running the function loop over columns
for col in cols:
  getUnik(col)

Column --company_permalink || Unique values count = 11573 
Column --company_name || Unique values count = 11573 
Column --company_category_code || Unique values count = 43 
Column --company_country_code || Unique values count = 2 
Column --company_state_code || Unique values count = 50 
Column --company_region || Unique values count = 546 
Column --company_city || Unique values count = 1229 
Column --investor_permalink || Unique values count = 10552 
Column --investor_name || Unique values count = 10465 
Column --investor_category_code || Unique values count = 33 
Column --investor_country_code || Unique values count = 72 
Column --investor_state_code || Unique values count = 50 
Column --investor_region || Unique values count = 585 
Column --investor_city || Unique values count = 990 
Column --funding_round_type || Unique values count = 9 
Column --funded_at || Unique values count = 2808 
Column --funded_month || Unique values count = 192 
Column --funded_quarter || Unique values coun

In [9]:
##Convert 'str' category columns to 'category' datatype

toCategoryCols = [
    'funded_year', 'funded_quarter', 'funding_round_type',
    'investor_state_code', 'investor_country_code', 'investor_category_code',
    'company_state_code', 'company_category_code', 'company_country_code'    
    ]

def convCategory(col):
  cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
  cnt=1
  mem_usage=list()

  for ch in cbase_iter:
    ##Convert 'str' category columns to 'category' datatype
    for col in toCategoryCols:
      ch[col] = ch[col].astype('category')

    ##Calculate the total memory footprint, and compare it with the previous one.
    mem=(ch.memory_usage(deep=True).sum()/(1024*1024))
    mem_usage.append(mem)
    print(f"Chunk -- {cnt} || Total memory used[Mb] -- {mem}")
    cnt+=1

  print(f"\nTotal memory used: {sum(mem_usage)}")
  
##Calling the function...
convCategory(col)

Chunk -- 1 || Total memory used[Mb] -- 3.383975028991699
Chunk -- 2 || Total memory used[Mb] -- 3.483393669128418
Chunk -- 3 || Total memory used[Mb] -- 3.4932451248168945
Chunk -- 4 || Total memory used[Mb] -- 3.4825706481933594
Chunk -- 5 || Total memory used[Mb] -- 3.492595672607422
Chunk -- 6 || Total memory used[Mb] -- 3.5080347061157227
Chunk -- 7 || Total memory used[Mb] -- 3.4949588775634766
Chunk -- 8 || Total memory used[Mb] -- 3.4686355590820312
Chunk -- 9 || Total memory used[Mb] -- 3.412302017211914
Chunk -- 10 || Total memory used[Mb] -- 3.062798500061035
Chunk -- 11 || Total memory used[Mb] -- 1.7644309997558594

Total memory used: 36.04694080352783


In [10]:
## clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
##Convert columns -> [funded_at, funded_month,	funded_quarter,	funded_year] to simpler integer type


def convDates():
  cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
  cnt=1
  mem_usage=list()

  for ch in cbase_iter:

    ch['funded_at']=ch['funded_at'].fillna('0')
    ch['funded_month']=ch['funded_month'].fillna('0')
    ch['funded_quarter']=ch['funded_quarter'].fillna('0')

    #convert column 'funded_at'
    ch['funded_at'] = [x.split('-')[-1] for x in ch['funded_at']]
    
    #convert column 'funded_month'
    ch['funded_month'] = [x.split('-')[-1] for x in ch['funded_month']]
    
    #convert column 'funded_quarter'
    ch['funded_quarter'] = [x.split('-')[-1] for x in ch['funded_quarter']]
  
    

    ##Calculate the total memory footprint, and compare it with the previous one.
    mem=(ch.memory_usage(deep=True).sum()/(1024*1024))
    mem_usage.append(mem)
    print(f"Chunk -- {cnt} || Total memory used[Mb] -- {mem}")
    cnt+=1

  #convert column 'funded_year'
  ch['funded_year'] = pd.to_numeric(ch['funded_year'], downcast='integer')
  ch['funded_at'] = pd.to_numeric(ch['funded_at'], downcast='integer')
  ch['funded_month']= pd.to_numeric(ch['funded_month'], downcast='integer')

  print(f"\nTotal memory used: {sum(mem_usage)}")
  
  
##Calling the function...
convDates()
     


Chunk -- 1 || Total memory used[Mb] -- 5.493410110473633
Chunk -- 2 || Total memory used[Mb] -- 5.442401885986328
Chunk -- 3 || Total memory used[Mb] -- 5.449219703674316
Chunk -- 4 || Total memory used[Mb] -- 5.4423627853393555
Chunk -- 5 || Total memory used[Mb] -- 5.438507080078125
Chunk -- 6 || Total memory used[Mb] -- 5.467597007751465
Chunk -- 7 || Total memory used[Mb] -- 5.445880889892578
Chunk -- 8 || Total memory used[Mb] -- 5.423828125
Chunk -- 9 || Total memory used[Mb] -- 5.310290336608887
Chunk -- 10 || Total memory used[Mb] -- 4.553667068481445
Chunk -- 11 || Total memory used[Mb] -- 2.614447593688965

Total memory used: 56.0816125869751


In [11]:
##Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


toCategoryCols = [
    'funded_year', 'funding_round_type',
    'investor_country_code', 'company_state_code', 'company_category_code', 'company_country_code'    
    ]


def optimizer():
  cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
  cnt=1
  mem_usage=list()

  for ch in cbase_iter:
    #drop columns with alot of missing values [investor_category_code,investor_state_code]
    ch.drop(['investor_category_code','investor_state_code'], axis=1, inplace=True)

    #Identify the numeric columns we can represent using more space efficient types
    ch['funded_year']= pd.to_numeric(ch['funded_year'], downcast='integer')

    #Convert 'str' category columns to 'category' datatype
    for col in toCategoryCols:
      ch[col] = ch[col].astype('category')

    #Convert columns -> [funded_at, funded_month,	funded_quarter,	funded_year] to simpler integer type
    ch['funded_at']=ch['funded_at'].fillna('0')
    ch['funded_month']=ch['funded_month'].fillna('0')
    ch['funded_quarter']=ch['funded_quarter'].fillna('0')

    #convert column 'funded_at'
    ch['funded_at'] = [x.split('-')[-1] for x in ch['funded_at']]
      
    #convert column 'funded_month'
    ch['funded_month'] = [x.split('-')[-1] for x in ch['funded_month']]
      
    #convert column 'funded_quarter'
    ch['funded_quarter'] = [x.split('-')[-1] for x in ch['funded_quarter']]

    #convert columns 'funded_*'
    ch['funded_year'] = pd.to_numeric(ch['funded_year'], downcast='integer')
    ch['funded_at'] = pd.to_numeric(ch['funded_at'], downcast='integer')
    ch['funded_month']= pd.to_numeric(ch['funded_month'], downcast='integer')

    ##Calculate the total memory footprint, and compare it with the previous one.
    mem=(ch.memory_usage(deep=True).sum()/(1024*1024))
    mem_usage.append(mem)
    print(f"Chunk -- {cnt} || Total memory used[Mb] -- {mem}")
    cnt+=1
  print(f"Total memory used by all chunks: {sum(mem_usage)}")

##calling the function...
optimizer()
     

Chunk -- 1 || Total memory used[Mb] -- 3.027604103088379
Chunk -- 2 || Total memory used[Mb] -- 3.130417823791504
Chunk -- 3 || Total memory used[Mb] -- 3.1404380798339844
Chunk -- 4 || Total memory used[Mb] -- 3.1298446655273438
Chunk -- 5 || Total memory used[Mb] -- 3.139590263366699
Chunk -- 6 || Total memory used[Mb] -- 3.155191421508789
Chunk -- 7 || Total memory used[Mb] -- 3.1708946228027344
Chunk -- 8 || Total memory used[Mb] -- 3.115872383117676
Chunk -- 9 || Total memory used[Mb] -- 3.059680938720703
Chunk -- 10 || Total memory used[Mb] -- 2.713253974914551
Chunk -- 11 || Total memory used[Mb] -- 1.5613937377929688
Total memory used by all chunks: 32.34418201446533


## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [13]:
# Your code goes here
#Create and connect to a new SQLite database file
conn = sqlite3.connect('cbase.db')

In [14]:
#Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.


def optimizer():
  cbase_iter = pd.read_csv(url, chunksize=csize, encoding= 'unicode_escape')
  cnt=1
  mem_usage=list()

  for ch in cbase_iter:
    #drop columns with alot of missing values [investor_category_code,investor_state_code]
    ch.drop(['investor_category_code','investor_state_code'], axis=1, inplace=True)

    #Identify the numeric columns we can represent using more space efficient types
    ch['funded_year']= pd.to_numeric(ch['funded_year'], downcast='integer')

    #Convert 'str' category columns to 'category' datatype
    for col in toCategoryCols:
      ch[col] = ch[col].astype('category')

    #Convert columns -> [funded_at, funded_month,	funded_quarter,	funded_year] to simpler integer type
    ch['funded_at']=ch['funded_at'].fillna('0')
    ch['funded_month']=ch['funded_month'].fillna('0')
    ch['funded_quarter']=ch['funded_quarter'].fillna('0')

    #convert column 'funded_at'
    ch['funded_at'] = [x.split('-')[-1] for x in ch['funded_at']]
      
    #convert column 'funded_month'
    ch['funded_month'] = [x.split('-')[-1] for x in ch['funded_month']]
      
    #convert column 'funded_quarter'
    ch['funded_quarter'] = [x.split('-')[-1] for x in ch['funded_quarter']]

    #convert columns 'funded_*'
    ch['funded_year'] = pd.to_numeric(ch['funded_year'], downcast='integer')
    ch['funded_at'] = pd.to_numeric(ch['funded_at'], downcast='integer')
    ch['funded_month']= pd.to_numeric(ch['funded_month'], downcast='integer')

    #Load to sql....!
    ch.to_sql(table, conn, if_exists='append', index=False)

    ##Calculate the total memory footprint, and compare it with the previous one.
    mem=(ch.memory_usage(deep=True).sum()/(1024*1024))
    mem_usage.append(mem)
    print(f"Chunk -- {cnt} || Total memory used[Mb] -- {mem}")
    cnt+=1
  print(f"Total memory used by all chunks: {sum(mem_usage)}")

optimizer()



Chunk -- 1 || Total memory used[Mb] -- 3.0280065536499023
Chunk -- 2 || Total memory used[Mb] -- 3.130824089050293
Chunk -- 3 || Total memory used[Mb] -- 3.141141891479492
Chunk -- 4 || Total memory used[Mb] -- 3.130420684814453
Chunk -- 5 || Total memory used[Mb] -- 3.139820098876953
Chunk -- 6 || Total memory used[Mb] -- 3.155278205871582
Chunk -- 7 || Total memory used[Mb] -- 3.171231269836426
Chunk -- 8 || Total memory used[Mb] -- 3.1162071228027344
Chunk -- 9 || Total memory used[Mb] -- 3.059786796569824
Chunk -- 10 || Total memory used[Mb] -- 2.7134313583374023
Chunk -- 11 || Total memory used[Mb] -- 1.5614795684814453
Total memory used by all chunks: 32.34762763977051


In [15]:
#Query the table and make sure the data types match up to what you had in mind for each column

results_df = pd.read_sql(f"PRAGMA table_info({table});", conn)
print(results_df)

    cid                   name     type  notnull dflt_value  pk
0     0      company_permalink     TEXT        0       None   0
1     1           company_name     TEXT        0       None   0
2     2  company_category_code     TEXT        0       None   0
3     3   company_country_code     TEXT        0       None   0
4     4     company_state_code     TEXT        0       None   0
5     5         company_region     TEXT        0       None   0
6     6           company_city     TEXT        0       None   0
7     7     investor_permalink     TEXT        0       None   0
8     8          investor_name     TEXT        0       None   0
9     9  investor_country_code     TEXT        0       None   0
10   10        investor_region     TEXT        0       None   0
11   11          investor_city     TEXT        0       None   0
12   12     funding_round_type     TEXT        0       None   0
13   13              funded_at  INTEGER        0       None   0
14   14           funded_month  INTEGER 

## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [16]:
# Your code goes here
#What proportion of the total amount of funds did the top 10% raise?  

q='SELECT SUM(raised_amount_usd) AS TOTAL_RAISED, company_name FROM investments GROUP BY company_name ORDER BY TOTAL_RAISED DESC;'

top10 = pd.read_sql(q, conn)
top10['PROPORTION%'] = (top10['TOTAL_RAISED'] / top10['TOTAL_RAISED'].sum())*100
tmp = top10.nlargest(int(top10.shape[0] * 0.1), 'TOTAL_RAISED')
print(tmp)
print(f"\nProportion raised by top 10%=> {tmp['PROPORTION%'].sum()} %")

      TOTAL_RAISED  company_name  PROPORTION%
0     2.968000e+10     Clearwire     4.353616
1     1.018540e+10       Groupon     1.494047
2     4.505000e+09     Nanosolar     0.660817
3     4.154100e+09      Facebook     0.609345
4     3.250000e+09  SurveyMonkey     0.476727
...            ...           ...          ...
1152  1.370000e+08     PlayFirst     0.020096
1153  1.370000e+08    Sittercity     0.020096
1154  1.366000e+08       Dropcam     0.020037
1155  1.365000e+08     BillFloat     0.020023
1156  1.365000e+08        Ruckus     0.020023

[1157 rows x 3 columns]

Proportion raised by top 10%=> 67.12772709906496 %


In [17]:
#What proportion of the total amount of funds did the top 1% raise?

tmp = top10.nlargest(int(top10.shape[0] * 0.01), 'TOTAL_RAISED')

print(f"\nProportion raised by top 1%=> {tmp['PROPORTION%'].sum()} %")


Proportion raised by top 1%=> 26.217737281766546 %


In [18]:
#What proportion of the total amount of funds did the bottom 10% raise

q='SELECT SUM(raised_amount_usd) AS TOTAL_RAISED, company_name FROM investments GROUP BY company_name ORDER BY TOTAL_RAISED ASC;'

bottom10 = pd.read_sql(q, conn)
bottom10['PROPORTION%'] = (bottom10['TOTAL_RAISED'] / bottom10['TOTAL_RAISED'].sum())*100

tmp = bottom10.nsmallest(int(bottom10.shape[0] * 0.1), 'TOTAL_RAISED')
print(tmp)
print(f"\nProportion raised by bottom 10%=> {tmp['PROPORTION%'].sum()} %")

      TOTAL_RAISED        company_name   PROPORTION%
1217        1000.0   Main Street Stark  1.466852e-07
1218        2000.0            uromovie  2.933703e-07
1219        3000.0          WhiteWilly  4.400555e-07
1220        3100.0            IndyGeek  4.547240e-07
1221        4000.0  PictureMe Universe  5.867407e-07
...            ...                 ...           ...
2369      675000.0             Kumbuya  9.901249e-05
2370      675000.0              Lottay  9.901249e-05
2371      675000.0     The Echo System  9.901249e-05
2372      675000.0      The Shared Web  9.901249e-05
2373      680000.0            ProBueno  9.974591e-05

[1157 rows x 3 columns]

Proportion raised by bottom 10%=> 0.03699021880568552 %


In [19]:
#What proportion of the total amount of funds did the bottom 1% raise

tmp = bottom10.nsmallest(int(bottom10.shape[0] * 0.01), 'TOTAL_RAISED')

print(f"\nProportion raised by bottom 1%=> {tmp['PROPORTION%'].sum()} %")


Proportion raised by bottom 1%=> 0.000252459841322347 %


In [20]:
#Which category of company attracted the most investments?

q='SELECT SUM(raised_amount_usd) AS TOTAL_RAISED, company_category_code FROM investments GROUP BY company_category_code ORDER BY TOTAL_RAISED DESC;'

top_category = pd.read_sql(q, conn)
top_category['PROPORTION%'] = (top_category['TOTAL_RAISED'] / top_category['TOTAL_RAISED'].sum())*100

tmp = top_category.nlargest(5, 'TOTAL_RAISED')
print("\nWhich category of company attracted the most investment? Below the Top 10 ... \n")
tmp


Which category of company attracted the most investment? Below the Top 10 ... 



,TOTAL_RAISED,company_category_code,PROPORTION%
0,1.103964e+11,biotech,16.193518
1,7.308452e+10,software,10.720415
2,6.477738e+10,mobile,9.501881
3,5.270523e+10,cleantech,7.731075
4,4.586093e+10,enterprise,6.727118


In [24]:
#Which investor contributed the most money (across all startups)?

q='SELECT SUM(raised_amount_usd) AS TOTAL_RAISED, investor_name FROM investments GROUP BY investor_name;'

top_investor = pd.read_sql(q, conn)
top_investor['PROPORTION%'] = (top_investor['TOTAL_RAISED'] / top_investor['TOTAL_RAISED'].sum())*100

top= top_investor.nlargest(10, 'TOTAL_RAISED')
print("\nWhich investor contributed the most money (across all startups)? => [Kleiner Perkins Caufield & Byers] \n ")
top


Which investor contributed the most money (across all startups)? => [Kleiner Perkins Caufield & Byers] 
 


,TOTAL_RAISED,investor_name,PROPORTION%
5375,1.121783e+10,Kleiner Perkins Caufield & Byers,1.645489
6689,9.692542e+09,New Enterprise Associates,1.421752
165,6.472126e+09,Accel Partners,0.949365
3709,6.375459e+09,Goldman Sachs,0.935185
8480,6.039402e+09,Sequoia Capital,0.885891
4356,5.969200e+09,Intel,0.875593
3716,5.808800e+09,Google,0.852065
9483,5.730000e+09,Time Warner,0.840506
2064,5.669000e+09,Comcast,0.831558
3824,4.960983e+09,Greylock Partners,0.727703


In [25]:
#Which investors contributed the most money per startup?

q='SELECT SUM(raised_amount_usd) AS TOTAL_RAISED, investor_name FROM investments GROUP BY company_name;'

top_invest2 = pd.read_sql(q, conn)
top_invest2['PROPORTION%'] = (top_invest2['TOTAL_RAISED'] / top_invest2['TOTAL_RAISED'].sum())*100

top = top_invest2.nlargest(10, 'TOTAL_RAISED')
print("\nBelow top10 investors who contributed the most money per startup?  \n ")
top


Below top10 investors who contributed the most money per startup?  
 


,TOTAL_RAISED,investor_name,PROPORTION%
2045,2.968000e+10,Eagle River Holdings,4.353616
4101,1.018540e+10,Ted Leonsis,1.494047
6257,4.505000e+09,Christian Reitberger,0.660817
3357,4.154100e+09,Reid Hoffman,0.609345
9330,3.250000e+09,Dave Goldberg,0.476727
11143,2.886013e+09,Reid Hoffman,0.423335
3505,2.788000e+09,U.S. Department of Energy,0.408958
2895,2.764400e+09,Pejman Nozad,0.405496
5398,2.685000e+09,Steve Case,0.393850
11507,2.600000e+09,Marlin Equity Partners,0.381381


In [26]:
#Which funding round was the most popular? 

q='SELECT COUNT(funding_round_type) AS ROUND_COUNTS, funding_round_type FROM investments GROUP BY funding_round_type;'

top_type = pd.read_sql(q, conn)

top = top_type.nlargest(10, 'ROUND_COUNTS')
print("\nBelow the most popular funding rounds ... \n ")
top
     


Below the most popular funding rounds ... 
 


,ROUND_COUNTS,funding_round_type
6,13938,series-a
8,10870,series-c+
1,8989,angel
9,8917,venture
7,8794,series-b
3,964,other
5,357,private-equity
4,33,post-ipo
2,5,crowdfunding
0,0,None


In [27]:
#Which was the least popular?

top = top_type.nsmallest(5, 'ROUND_COUNTS')
print("\nBelow the least popular funding rounds ... \n ")
top


Below the least popular funding rounds ... 
 


,ROUND_COUNTS,funding_round_type
0,0,None
2,5,crowdfunding
4,33,post-ipo
5,357,private-equity
3,964,other
